In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import time
import yfinance as yf
import os
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import os
from os import listdir
from os.path import isfile, join
import warnings
warnings.simplefilter("ignore")

In [2]:
PATH = "/home/carlosdomiguez/Proyectos/Python/fast-finance/stocks/"
S_DATE = "2017-01-01"
E_DATE = "2022-12-06"
S_DATE_DT = pd.to_datetime(S_DATE)
E_DATE_DT = pd.to_datetime(E_DATE)

In [3]:
def get_column_from_csv(file, col_name):
    try:
        df = pd.read_csv(file)
    except:
        print("File doesnt exist")
    else:
        return df[col_name]

In [4]:
tickers = get_column_from_csv("/home/carlosdomiguez/Proyectos/Python/fast-finance/5000_Stocks.csv", "Ticker")
print(len(tickers))

3481


In [5]:
def save_to_csv_from_yahoo(folder, ticker):
    stock = yf.Ticker(ticker)
    try:
        print("Getting data for: ", ticker)
        df = stock.history(period="5y")
        time.sleep(1)
        the_file = folder+ticker.replace(".", "_")+'.csv'
        df.to_csv(the_file)
        print(the_file, " Saved")
    except Exception as ex:
        print("Couldnt get data for: ", ticker)

In [6]:
# for x in range(0, 3481):
#     save_to_csv_from_yahoo(PATH, tickers[x])
# print("Finished")

In [7]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH+ticker+'.csv', index_col=0)
    except FileNotFoundError:
        print("File not exists")
    else:
        return df

In [8]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers.sort()

In [9]:
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close']/df['Close'].shift(1))-1
    # df.to_csv(PATH+ticker+'.csv')
    return df

In [10]:
def add_cum_return_to_df(df):
    df['daily_return'] = (1 + df['daily_return']).cumprod()
    return df

In [11]:
def add_bollinger_bands(df):
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    # df.to_csv(PATH+ticker+'.csv')
    return df

In [12]:
def add_ichimoku(df):
    high_val=df['High'].rolling(window=9).max()
    low_val=df['High'].rolling(window=9).min()
    df['Conversion']=(high_val+low_val)/2
    high_val2=df['High'].rolling(window=26).max()
    low_val2=df['High'].rolling(window=26).min()
    df['Baseline']=(high_val2+low_val2)/2
    df['SpanA']=((df['Conversion']+df['Baseline'])/2)
    high_val3=df['High'].rolling(window=26).max()
    low_val3=df['High'].rolling(window=26).min()
    df['SpanB']=((high_val3+low_val3)/2).shift(-26)
    df['Lagging']=df['Close'].shift(-26)
    return df

In [13]:
# for tick in tickers:
#     try:
#         print("Working on: ", tick)
#         new_df = get_stock_df_from_csv(tick)
#         new_df = add_daily_return_to_df(new_df)
#         new_df = add_cum_return_to_df(new_df)
#         new_df = add_bollinger_bands(new_df)
#         new_df = add_ichimoku(new_df)
#         new_df.to_csv(PATH+tick+'.csv')
#     except Exception as ex:
#         print(ex)

In [14]:
def plot_with_boll_bands(df, ticker):
    fig = go.Figure()
    candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='CandleStick')
    upper_line = go.Scatter(x=df.index, y=df['upper_band'], line=dict(color='rgba(250, 0, 0, 0.75)', width=1), name='UpperBand')
    middle_line = go.Scatter(x=df.index, y=df['middle_band'], line=dict(color='rgba(0, 0, 250, 0.75)', width=0.7), name='MiddleBand')
    lower_line = go.Scatter(x=df.index, y=df['lower_band'], line=dict(color='rgba(0, 250, 0, 0.75)', width=1), name='LowerBand')
    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(middle_line)
    fig.add_trace(lower_line)
    fig.update_xaxes(title="Date", rangeslider_visible=True)
    fig.update_yaxes(title="Price")
    fig.update_layout(title=ticker+" Bollinger Bands", height=1200, width=1800, showlegend=True)
    fig.show()

In [15]:
    bollings_test = "MSFT"
    test = get_stock_df_from_csv(bollings_test)
    plot_with_boll_bands(test, bollings_test)

In [16]:
def get_fill_color(label):
    if label >= 1:
        return 'rgba(0, 250, 0, 0.4)'
    else:
        return 'rgba(250, 0, 0, 0.4)'

In [17]:
def get_ichimoku(df):

    candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df["Low"], close=df['Close'], name="Candlestick")
    df1 = df.copy()
    fig = go.Figure()
    df['label'] = np.where(df['SpanA'] > df['SpanB'], 1, 0)
    df['group'] = df['label'].ne(df['label'].shift()).cumsum()
    df = df.groupby('group')

    dfs = []
    for name, data in df:
        dfs.append(data)
    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index, y=df.SpanA, line=dict(color='rgba(0,0,0,0)')))
        fig.add_traces(go.Scatter(x=df.index, y=df.SpanB, line=dict(color='rgba(0,0,0,0)'), fill='tonexty', fillcolor=get_fill_color(df['label'].iloc[0])))
    baseline = go.Scatter(x=df1.index, y=df1['Baseline'], line=dict(color='pink', width=2), name="Baseline")
    conversion = go.Scatter(x=df1.index, y=df1['Conversion'], line=dict(color='black', width=1), name="Conversion")
    lagging = go.Scatter(x=df1.index, y=df1['Lagging'], line=dict(color='purple', width=2), name="Lagging")
    span_a = go.Scatter(x=df1.index, y=df1['SpanA'], line=dict(color='green', width=2, dash='dot'), name="Span A")
    span_b = go.Scatter(x=df1.index, y=df1['SpanB'], line=dict(color='red', width=1, dash='dot'), name="Span B")
    fig.add_trace(candle)
    fig.add_trace(baseline)     #kijun-sen      pink
    fig.add_trace(conversion)   #tenkan-sen     black
    fig.add_trace(lagging)      #chikou         purple
    fig.add_trace(span_a)       #senkouA        green dotted
    fig.add_trace(span_b)       #senkeouB       red dotted
    fig.update_layout(height=1200, width=1800, showlegend=True)
    fig.show()

In [19]:
ichimoku_ticker = "TSLA"
test_ichimoku = get_stock_df_from_csv(ichimoku_ticker)
get_ichimoku(test_ichimoku)